# <span style="color:green"> Import libraries<span style="color:green">

In [1]:
#Import all the required libraries
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn 
import random
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras.layers import Input, Dense, LeakyReLU, ReLU
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
from sklearn.feature_selection import f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pybiomart
from biomart import BiomartServer
from sklearn.feature_selection import RFECV
import xgboost as xgb

2024-05-17 15:56:09.091752: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 15:56:09.091815: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 15:56:09.092846: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-17 15:56:09.102346: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-17 15:56:10.072640: W tensorflow/compiler/tf2

# <span style="color:green"> ML for Feature Selection and Association <span style="color:green">

In [2]:
#Read the file 
df= pd.read_table("/home/hapmap/240305193625/Plink/GWAStutorialldl_newraw.raw", delim_whitespace=False)
# print(df.head())
# print("Shape of file", df.shape)

In [3]:
df.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,rs2905036_T,rs4245756_C,rs7537756_A,rs6694982_A,...,rs6644898_G,rs34605807_A,rs28600179_G,rs28655329_C,rs5982824_G,rs5939299_C,rs28814596_C,rs17653586_C,rs5983831_G,rs7876575_C
0,1,NA19919,-9,-9,1,1643.12,2,2,2,2,...,0,0,2,1,0,2,2,2,2,2
1,1,NA19916,-9,-9,1,1622.42,1,2,2,2,...,0,1,1,2,1,1,2,2,1,2
2,1,NA19835,-9,-9,1,1603.07,2,2,2,1,...,2,0,2,2,1,2,2,2,2,2
3,1,NA20282,-9,-9,1,1634.15,2,2,1,1,...,1,1,2,1,1,1,2,2,2,2
4,1,NA19703,-9,-9,1,1623.19,2,2,1,2,...,1,1,2,1,1,2,2,2,2,2


In [4]:
new_col_names= list(df.columns[:6])
new_col_names.extend([col_name[:-2] for col_name in df.columns[6:]])
df.columns=new_col_names
df.columns

Index(['FID', 'IID', 'PAT', 'MAT', 'SEX', 'PHENOTYPE', 'rs2905036',
       'rs4245756', 'rs7537756', 'rs6694982',
       ...
       'rs6644898', 'rs34605807', 'rs28600179', 'rs28655329', 'rs5982824',
       'rs5939299', 'rs28814596', 'rs17653586', 'rs5983831', 'rs7876575'],
      dtype='object', length=184515)

In [5]:
# #X is Independent feature and y is dependent feature
X= df.iloc[:, 6:]
print("Shape of X", X.shape)
y= df.PHENOTYPE
print("Shape of y", y.shape)
X

Shape of X (1397, 184509)
Shape of y (1397,)


,rs2905036,rs4245756,rs7537756,rs6694982,rs3748592,rs3748594,rs28391282,rs9777703,rs35940137,rs9697358,...,rs6644898,rs34605807,rs28600179,rs28655329,rs5982824,rs5939299,rs28814596,rs17653586,rs5983831,rs7876575
0,2,2,2,2,2,2,2,1,2,2,...,0,0,2,1,0,2,2,2,2,2
1,1,2,2,2,2,2,2,2,2,2,...,0,1,1,2,1,1,2,2,1,2
2,2,2,2,1,2,2,1,0,2,2,...,2,0,2,2,1,2,2,2,2,2
3,2,2,1,1,2,2,2,1,2,2,...,1,1,2,1,1,1,2,2,2,2
4,2,2,1,2,2,1,2,1,2,2,...,1,1,2,1,1,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1392,2,1,0,2,2,2,2,1,2,2,...,2,2,2,1,2,1,2,2,2,2
1393,2,1,1,2,1,2,1,2,2,2,...,2,1,0,1,1,1,2,2,2,2
1394,2,2,1,2,2,2,1,1,2,1,...,1,1,2,0,1,1,2,2,2,2
1395,2,2,0,2,2,2,1,2,2,0,...,1,1,2,2,1,2,2,2,1,2


In [6]:
#Train test data split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=2000)
print("The dimension of X_train is {}".format(X_train.shape))
print("The dimension of X_test is {}".format(X_test.shape))

The dimension of X_train is (977, 184509)
The dimension of X_test is (420, 184509)


In [7]:
#Standardization of X_train and X_test data by Standard Scaler
scaler= StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)

In [8]:
y

0       1643.12
1       1622.42
2       1603.07
3       1634.15
4       1623.19
         ...   
1392    1614.51
1393    1613.63
1394    1622.67
1395    1601.58
1396    1639.02
Name: PHENOTYPE, Length: 1397, dtype: float64

## <span style="color:green"> LASSO <span style="color:green">

In [9]:
# Create and fit the LASSOCV model select 5000 SNP
lasso_cv = linear_model.LassoCV(alphas= [0.0009], cv=5, random_state=2000, n_jobs=70)

# Fit the model to the data
lasso_cv.fit(X,y)

#Print the optimal alpha value
print ("Optimal alpha:", lasso_cv.alpha_)

# Indices of the SNPs with non-zero coefficients
lasso_snps_ldl = np.where(lasso_cv.coef_)[0]

# See how many SNPs have a non-zero coefficient
print(len(lasso_snps_ldl), "selected SNPs")

#Final SNPs
l1= X.iloc[:,lasso_snps_ldl]
print("print shape of selected snps by lasso",l1.shape)


/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.513e+02, tolerance: 1.412e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.558e+02, tolerance: 1.374e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Du

Optimal alpha: 0.0009
5006 selected SNPs
print shape of selected snps by lasso (1397, 5006)


### Linear regression

In [10]:
Y= df.PHENOTYPE
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(l1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

# f_statistic, p_value = f_regression(l1, Y)
# selected_p_indices = np.where(p_value < 5e-04)[0]
# selected_p_values = p_value[selected_p_indices]
# print("Selected p-values:")
# print(selected_p_values)
# print("Number of selected features:", len(selected_p_indices))
# selected_f_values = f_statistic[selected_p_indices]
# print("Selected F-values:")
# print(selected_f_values)
# top_f_indices = np.argsort(-selected_f_values)
# print("Indices of selected features with top F-values:")
# print(top_f_indices)
# top_f_features = l1.iloc[:, selected_p_indices[top_f_indices]]
# print("Top selected features:")
# print(top_f_features)
# lasso_LR_sigSNP=l1.iloc[:, selected_p_indices]
# lasso_LR_sigSNP
# lasso_LR_sig_col= list(lasso_LR_sigSNP.columns)
# lasso_LR_sig_col

Y.shape (1397,)
x_train=(977, 5006), x_test=(420, 5006), Y_train= (977,), Y_test= (420,)
r2 is  0.5791382482551777
mean_absolute_error 12.465512537052506
mean_squared_error  252.84025371490713
explained_variance_score 0.782657089765626


### Random Forest

In [11]:
# Create a Random Forest Regressor
 #reate a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=10,random_state=2000)

# Create RFECV object
# rfecv = RFECV(estimator=rf_regressor, step=30, cv=5, scoring='neg_mean_squared_error', n_jobs=20)

# # Fit RFECV on the training data
# rfecv.fit(x_train, Y_train)

# # Manually set the number of features you want to keep
# desired_num_features =100
# # Manually set the number of features you want to keep


# # Get the selected features based on the desired number
# selected_features = rfecv.get_support(indices=True)[:desired_num_features]
# # Print the indices of the selected features
# print("Selected features indices:", selected_features)

# # Transform the training and testing data to keep only the selected features
# X_train_selected = rfecv.transform(x_train)
# X_test_selected = rfecv.transform(x_test)


# # Get the optimal number of features
# optimal_num_features = rfecv.n_features_

# # Print the optimal number of features
# #print(f"Optimal number of features: {optimal_num_features}")


# # Train a Random Forest Regressor on the selected features
# rf_regressor_selected = RandomForestRegressor(n_estimators=100, random_state=2000)
rf_regressor.fit(x_train, Y_train)

# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))
# lasso_RF_sigSNP=l1.iloc[:,selected_features]
# lasso_RF_sig_col= list(lasso_RF_sigSNP.columns)
# lasso_RF_sig_col

r2 is  -1.1080667953243561
mean_absolute_error 20.644516940231085
mean_squared_error  697.633794355837
explained_variance_score 0.40026529374490283


### Support Vector Machine

In [12]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))
# perm_importance= permutation_importance(svr, x_test, Y_test, n_repeats=10, random_state=2000, n_jobs=20)


# imp_score=perm_importance.importances_mean
# feature_indices = np.argsort(imp_score)[::-1]
# print(f"Selected feature indices are {feature_indices}")

# # # Specify the number of features you want to select based on magnitude
# num_selected_features = 100
# top_100=feature_indices[:num_selected_features]
# print(f"Top 100 fearure indices are {feature_indices}")
# imp_score_100=imp_score[top_100]
# print(f"Top 100 fearures permutation scores are {imp_score_100}")
# lasso_SVR_sigSNP= l1.iloc[:,top_100]
# lasso_SVR_sig_col= list(lasso_SVR_sigSNP.columns)
# lasso_SVR_sig_col


r2 is  0.721099901900792
mean_absolute_error 12.161761948920846
mean_squared_error  239.6606379732451
explained_variance_score 0.7938640460916601


### XGBoost

In [13]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 score",score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -0.7567837899784347
mean_absolute_error 21.47790563383557
mean_squared_error  758.2477771266286
explained_variance_score 0.3489553142514311


## <span style="color:green"> Ridge<span style="color:green">

In [14]:
# Create and fit the RidgeCV model select 5000 SNP
ridge_cv = linear_model.RidgeCV(alphas=[0.005], cv=5, scoring='neg_mean_squared_error')
ridge_cv.fit(X, y)

# Optimal alpha value
print("Optimal alpha:", ridge_cv.alpha_)

# Indices of the SNPs with non-zero coefficients
ridge_snps_ldl = np.where(ridge_cv.coef_)[0]

# Select coefficients with non-zero values
selected_features = ridge_cv.coef_ != 0

# Select coefficients of selected SNPs
coefficients = ridge_cv.coef_[selected_features]

# Get the indices of the top 5000 SNPs based on descending coefficients
top_n = 5000
sorted_indices = np.argsort(-coefficients)
selected_indices = sorted_indices[:top_n]

# Print the indices of selected SNPs and the number of selected SNPs
print("Selected SNPs indices based on top coefficients:", selected_indices)
print(len(selected_indices), "selected SNPs")

#Maximum Coefficient
print("Maximum value of coefficient", coefficients.max())

#Final SNPs
R1= X.iloc[:,selected_indices]
print("print shape of selected snps by Ridge",R1.shape)


Optimal alpha: 0.005
Selected SNPs indices based on top coefficients: [ 49952 181233  21114 ... 178888 156978 130250]
5000 selected SNPs
Maximum value of coefficient 0.5110441437104237
print shape of selected snps by Ridge (1397, 5000)


### Linear Regression

In [15]:
Y= df.PHENOTYPE
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(R1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (1397,)
x_train=(977, 5000), x_test=(420, 5000), Y_train= (977,), Y_test= (420,)


In [16]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

# f_statistic, p_value = f_regression(R1, Y)
# selected_p_indices = np.where(p_value < 5e-04)[0]
# selected_p_values = p_value[selected_p_indices]
# print("Selected p-values:")
# print(selected_p_values)
# print("Number of selected features:", len(selected_p_indices))
# selected_f_values = f_statistic[selected_p_indices]
# print("Selected F-values:")
# print(selected_f_values)
# top_f_indices = np.argsort(-selected_f_values)
# print("Indices of selected features with top F-values:")
# print(top_f_indices)
# top_f_features = R1.iloc[:, selected_p_indices[top_f_indices]]
# print("Top selected features:")
# print(top_f_features)
# Ridge_LR_sigSNP=R1.iloc[:, selected_p_indices]
# Ridge_LR_sigSNP
# Ridge_LR_sig_col= list(Ridge_LR_sigSNP.columns)
# Ridge_LR_sig_col

r2 is  0.7495365521265533
mean_absolute_error 10.571163767740806
mean_squared_error  184.09266646612926
explained_variance_score 0.8419250391940631


### Random Forest

In [17]:
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=20,random_state=2000)

# Create RFECV object
# rfecv = RFECV(estimator=rf_regressor, step=30, cv=5, scoring='neg_mean_squared_error', n_jobs=4)

# # Fit RFECV on the training data
# rfecv.fit(x_train, Y_train)

# # Manually set the number of features you want to keep
# desired_num_features =100
# # Manually set the number of features you want to keep


# # Get the selected features based on the desired number
# selected_features = rfecv.get_support(indices=True)[:desired_num_features]
# # Print the indices of the selected features
# print("Selected features indices:", selected_features)

# # Transform the training and testing data to keep only the selected features
# X_train_selected = rfecv.transform(x_train)[:, selected_features]
# X_test_selected = rfecv.transform(x_test)[:, selected_features]


# # Get the optimal number of features
# optimal_num_features = rfecv.n_features_

# # Print the optimal number of features
#print(f"Optimal number of features: {optimal_num_features}")


# Train a Random Forest Regressor on the selected features
# rf_regressor_selected = RandomForestRegressor(n_estimators=100, random_state=2000)
rf_regressor.fit(x_train, Y_train)

# Make predictions on the test set using the model with selected features
y_pred = rf_regressor.predict(x_test)

score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))
# Ridge_RF_sigSNP=R1.iloc[:,selected_features]
# Ridge_RF_sig_col= list(Ridge_RF_sigSNP.columns)
# Ridge_RF_sig_col

r2 is  -25.458692490331657
mean_absolute_error 24.130327097668317
mean_squared_error  1037.776645500237
explained_variance_score 0.10813911518407104


### Support Vector Machine

In [18]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))
# perm_importance= permutation_importance(svr, x_test, Y_test, n_repeats=10, random_state=2000, n_jobs=10)


# imp_score=perm_importance.importances_mean
# feature_indices = np.argsort(np.abs(imp_score))[::-1]
# print(f"Selected feature indices are {feature_indices}")

# # # Specify the number of features you want to select based on magnitude
# num_selected_features = 100
# top_100=feature_indices[:num_selected_features]
# print(f"Top 100 fearure indices are {feature_indices}")
# imp_score_100=imp_score[top_100]
# print(f"Top 100 fearures permutation scores are {imp_score_100}")

# Ridge_SVR_sigSNP= R1.iloc[:,top_100]
# Ridge_SVR_sig_col= list(Ridge_SVR_sigSNP.columns)
# Ridge_SVR_sig_col

r2 is  0.7444089301350973
mean_absolute_error 12.862723241150734
mean_squared_error  265.44715405167744
explained_variance_score 0.7720143400428139


### XGboost

In [19]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 score",score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -6.518809301431905
mean_absolute_error 25.53861571103051
mean_squared_error  1113.4154745956616
explained_variance_score 0.04489672489733043


## <span style="color:green"> Elastic net<span style="color:green">

In [20]:
# Create and fit the Elasticnet model
elastic_cv = linear_model.ElasticNetCV(l1_ratio=[0.5],alphas=[0.09], cv=5, random_state= 2000, n_jobs=70)
elastic_cv.fit(X, y)

# Print the optimal alpha value
print ("Optimal alpha:", elastic_cv.alpha_)
print("Optimal l1_ratio: ", elastic_cv.l1_ratio_)

# Indices of the SNPs with non-zero coefficients
elastic_snps_ldl = np.where(elastic_cv.coef_)[0]

# See how many SNPs have a non-zero coefficient
print(len(elastic_snps_ldl), "selected SNPs")

#Final SNPs
E1= X.iloc[:,elastic_snps_ldl]
print("print shape of selected snps by Elastic net",E1.shape)

/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.644e+03, tolerance: 1.374e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.699e+03, tolerance: 1.443e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Du

Optimal alpha: 0.09
Optimal l1_ratio:  0.5
5234 selected SNPs
print shape of selected snps by Elastic net (1397, 5234)


### Linear Regression

In [21]:
Y= df.PHENOTYPE
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(E1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {x_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (1397,)
x_train=(977, 5234), x_test=(420, 5234), Y_train= (977, 5234), Y_test= (420,)


In [22]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

# f_statistic, p_value = f_regression(E1, Y)
# selected_p_indices = np.where(p_value < 5e-04)[0]
# selected_p_values = p_value[selected_p_indices]
# print("Selected p-values:")
# print(selected_p_values)
# print("Number of selected features:", len(selected_p_indices))
# selected_f_values = f_statistic[selected_p_indices]
# print("Selected F-values:")
# print(selected_f_values)
# top_f_indices = np.argsort(-selected_f_values)
# print("Indices of selected features with top F-values:")
# print(top_f_indices)
# top_f_features = E1.iloc[:, selected_p_indices[top_f_indices]]
# print("Top selected features:")
# print(top_f_features)
# Elasticnet_LR_sigSNP=E1.iloc[:, selected_p_indices]
# Elasticnet_LR_sigSNP
# Elasticnet_LR_sig_col= list(Elasticnet_LR_sigSNP.columns)
# Elasticnet_LR_sig_col

r2 is  0.8942542279665269
mean_absolute_error 7.360016430608836
mean_squared_error  87.24261030890234
explained_variance_score 0.9250345355808147


### Random Forest

In [23]:
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=20,random_state=2000)

# # Create RFECV object
# rfecv = RFECV(estimator=rf_regressor, step=30, cv=5, scoring='neg_mean_squared_error', n_jobs=70)

# # Fit RFECV on the training data
# rfecv.fit(x_train, Y_train)

# # Manually set the number of features you want to keep
# desired_num_features =100
# # Manually set the number of features you want to keep


# # Get the selected features based on the desired number
# selected_features = rfecv.get_support(indices=True)[:desired_num_features]
# # Print the indices of the selected features
# print("Selected features indices:", selected_features)

# # Transform the training and testing data to keep only the selected features
# X_train_selected = rfecv.transform(x_train)[:, selected_features]
# X_test_selected = rfecv.transform(x_test)[:, selected_features]


# # Get the optimal number of features
# optimal_num_features = rfecv.n_features_

# # Print the optimal number of features
#print(f"Optimal number of features: {optimal_num_features}")


# Train a Random Forest Regressor on the selected features
# rf_regressor_selected = RandomForestRegressor(n_estimators=100, random_state=2000)
rf_regressor.fit(x_train, Y_train)

# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))
# Elasticnet_RF_sigSNP=E1.iloc[:,selected_features]
# Elasticnet_RF_sig_col= list(Elasticnet_RF_sigSNP.columns)
# Elasticnet_RF_sig_col

r2 is  -0.9569676130372007
mean_absolute_error 20.184452330021944
mean_squared_error  656.2532810317174
explained_variance_score 0.4355302096863958


### Support Vector Regression

In [24]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))
#perm_importance= permutation_importance(svr, x_test, Y_test, n_repeats=10, random_state=2000, n_jobs=70)


# imp_score=perm_importance.importances_mean
# feature_indices = np.argsort(np.abs(imp_score))[::-1]
# print(f"Selected feature indices are {feature_indices}")
# # # Specify the number of features you want to select based on magnitude
# num_selected_features = 100
# top_100=feature_indices[:num_selected_features]
# print(f"Top 100 fearure indices are {feature_indices}")
# imp_score_100=imp_score[top_100]
# print(f"Top 100 fearures permutation scores are {imp_score_100}")
# Elasticnet_SVR_sigSNP= E1.iloc[:,top_100]
# Elasticnet_SVR_sig_col= list(Elasticnet_SVR_sigSNP.columns)
# Elasticnet_SVR_sig_col

r2 is  0.9060954040455816
mean_absolute_error 7.782449770546358
mean_squared_error  98.74576826435285
explained_variance_score 0.9150635119925715


### XGBoost

In [25]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 score",score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 score -0.8781806156008922
mean_absolute_error 21.23531454613095
mean_squared_error  750.599068979581
explained_variance_score 0.3547242389716916


## <span style="color:green"> Mutual information <span style="color:green">

In [26]:
# Create and fit the Mutual_info model
mutual_info= mutual_info_regression(X, y, random_state=2000)
print("Length of mutal info", len(mutual_info))
mutual_info= pd.Series(mutual_info, index=df.columns[6:])

#Sort mutual_info values descending order
vdesc= mutual_info.sort_values(ascending=False)
vdesc_nonzero= vdesc[vdesc!=0]
print("Maximum value of mutualinfo", vdesc_nonzero.max)
print("Length of mutal info non zero", len(vdesc_nonzero))
k= np.where(vdesc_nonzero)
first_col= k[0]

#Selected 5000 SNPs
top5000 = 5000
mi_pos= first_col[:top5000]

#Final SNPs
M1= X.iloc[:,mi_pos]
print("print shape of selected snps by Mutual_info",M1.shape)

Length of mutal info 184509
Maximum value of mutualinfo <bound method NDFrame._add_numeric_operations.<locals>.max of rs35284732    1.076325e-01
rs8183216     1.076325e-01
rs9757258     1.076325e-01
rs2173077     1.076325e-01
rs11645945    1.076325e-01
                  ...     
rs7251578     9.338578e-07
rs9494623     8.373688e-07
rs1933517     6.334198e-07
rs17090150    7.810892e-08
rs16870291    2.609217e-08
Length: 107237, dtype: float64>
Length of mutal info non zero 107237
print shape of selected snps by Mutual_info (1397, 5000)


In [27]:
print("hello")

hello


### Linear Regression

In [28]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(M1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (1397,)
x_train=(977, 5000), x_test=(420, 5000), Y_train= (977,), Y_test= (420,)


In [29]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

# f_statistic, p_value = f_regression(M1, Y)
# selected_p_indices = np.where(p_value < 5e-04)[0]
# selected_p_values = p_value[selected_p_indices]
# print("Selected p-values:")
# print(selected_p_values)
# print("Number of selected features:", len(selected_p_indices))
# selected_f_values = f_statistic[selected_p_indices]
# print("Selected F-values:")
# print(selected_f_values)
# top_f_indices = np.argsort(-selected_f_values)
# print("Indices of selected features with top F-values:")
# print(top_f_indices)
# top_f_features = M1.iloc[:, selected_p_indices[top_f_indices]]
# print("Top selected features:")
# print(top_f_features)
# MI_LR_sigSNP=M1.iloc[:, selected_p_indices]
# MI_LR_sigSNP
# MI_LR_sig_col= list(MI_LR_sigSNP.columns)
# MI_LR_sig_col

r2 is  -1.6731668601292902
mean_absolute_error 32.10365117629098
mean_squared_error  1783.8521895339607
explained_variance_score -0.5325159636765415


### Random Forest

In [30]:
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

# Create RFECV object
# rfecv = RFECV(estimator=rf_regressor, step=30, cv=5, scoring='neg_mean_squared_error')

# # Fit RFECV on the training data
# rfecv.fit(x_train, Y_train)

# # Manually set the number of features you want to keep
# desired_num_features =100
# # Manually set the number of features you want to keep


# # Get the selected features based on the desired number
# selected_features = rfecv.support_
# # Print the indices of the selected features
# print("Selected features indices:", selected_features)

# # Transform the training and testing data to keep only the selected features
# X_train_selected = rfecv.transform(x_train)
# X_test_selected = rfecv.transform(x_test)


# # Get the optimal number of features
# optimal_num_features = rfecv.n_features_

# # Print the optimal number of features
# #print(f"Optimal number of features: {optimal_num_features}")


# Train a Random Forest Regressor on the selected features
#rf_regressor= RandomForestRegressor(n_estimators=100, random_state=2000)
rf_regressor.fit(x_train, Y_train)

# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)

score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))
# MI_RF_sigSNP=M1.iloc[:,selected_features]
# MI_RF_sig_col= list(MI_RF_sigSNP.columns)
# MI_RF_sig_col

r2 is  -27.231205947401545
mean_absolute_error 25.116006879717293
mean_squared_error  1133.080150880739
explained_variance_score 0.025369276531193474


### Support Vector Regressor

In [31]:
svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale')
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
mae= print(f"MAE is {mean_absolute_error(Y_test, y_pred)}")
# perm_importance= permutation_importance(svr, x_test, Y_test, n_repeats=10, random_state=2000, n_jobs=70)


# imp_score=perm_importance.importances_mean
# feature_indices = np.argsort(np.abs(imp_score))[::-1]
# print(f"Selected feature indices are {feature_indices}")

# # # Specify the number of features you want to select based on magnitude
# num_selected_features = 100
# top_100=feature_indices[:num_selected_features]
# print(f"Top 100 fearure indices are {feature_indices}")
# imp_score_100=imp_score[top_100]
# print(f"Top 100 fearures permutation scores are {imp_score_100}")
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))
# MI_SVR_sigSNP= M1.iloc[:,top_100]
# MI_SVR_sig_col= list(MI_SVR_sigSNP.columns)
# MI_SVR_sig_col

MAE is 31.809512084671944
r2 is  -1.946825585813118
mean_absolute_error 31.809512084671944
mean_squared_error  1765.1336316588079
explained_variance_score -0.5183092884378504


### XGBoost

In [32]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 score",score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -6.636541741017606
mean_absolute_error 26.3161754906064
mean_squared_error  1228.7418391355063
explained_variance_score -0.054511672815277246
